In [1]:
import pandas as pd
from ranker.ranker import Ranker
from ranker.ranker_factory import RankerFactory
from database.market import Market
from database.sec import SEC
from database.adatabase import ADatabase
from processor.processor import Processor as p
from datetime import datetime, timedelta
from tqdm import tqdm
import numpy as np
from modeler_strats.universal_modeler import UniversalModeler

In [2]:
## initializing ranker_class
ranker_class = RankerFactory.build(Ranker.WEEKLY_STOCK_ROLLING_RANKER)

In [3]:
backtest_start_date = datetime(2016,1,1)
backtest_end_date = datetime(2023,1,1)
current_year = datetime.now().year

In [4]:
## creating database objects
market = Market()
modeler_strat = UniversalModeler()

In [5]:
market.connect()
sp500 = market.retrieve("sp500")
market.disconnect()
sp500 = sp500.rename(columns={"Symbol":"ticker"})

In [6]:
market.connect()
training_sets = []
for ticker in tqdm(sp500["ticker"].unique()):
    try:
        prices = market.retrieve_ticker_prices(ranker_class.asset_class.value,ticker)
        prices = p.column_date_processing(prices)
        ticker_data = ranker_class.training_set(ticker,prices)
        training_sets.append(ticker_data)
    except Exception as e:
        print(str(e))
        continue
market.disconnect()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [00:33<00:00, 14.56it/s]


In [7]:
data = pd.concat(training_sets)

In [8]:
training_data = data.dropna().copy().sort_values(["year","week"])

In [9]:
training_data = training_data.groupby(["year","quarter","week","ticker"]).mean().reset_index()

In [10]:
training_data

,year,quarter,week,ticker,rank_prediction
0,2000,2,21,A,59.699765
1,2000,2,21,AAPL,0.869650
2,2000,2,21,ABC,3.346057
3,2000,2,21,ABMD,24.308350
4,2000,2,21,ABT,8.861870
...,...,...,...,...,...
544642,2023,2,23,YUM,131.573586
544643,2023,2,23,ZBH,129.178231
544644,2023,2,23,ZBRA,294.525350
544645,2023,2,23,ZION,36.999472


In [15]:
# ranker_class.db.connect()
# relevant_columns = list(set(list(training_data.columns)) - set(ranker_class.factors))
# ranker_class.db.store("sim",training_data[relevant_columns])
# ranker_class.db.disconnect()

In [12]:
ranker_class.db.connect()
ranker_class.db.drop("predictions")
relevant_columns = list(set(list(training_data.columns)) - set(ranker_class.factors))
predictions = ranker_class.sim_processor(training_data)
ranker_class.db.store("predictions",predictions[predictions["year"]==current_year][relevant_columns])
ranker_class.db.disconnect()

In [14]:
predictions[predictions["year"]==current_year][relevant_columns]

,week,rank_prediction,quarter,ticker,year
533427,2,138.303936,1,A,2023
533428,2,13.584425,1,AAL,2023
533429,2,163.121462,1,AAP,2023
533430,2,147.586776,1,AAPL,2023
533431,2,145.692922,1,ABBV,2023
...,...,...,...,...,...
544642,24,131.573586,2,YUM,2023
544643,24,129.178231,2,ZBH,2023
544644,24,294.525350,2,ZBRA,2023
544645,24,36.999472,2,ZION,2023
